Before starting we import all the libraries that we need

In [2]:
from MyFunctions.crawler import Crawler
from MyFunctions.parser import Parser
from MyFunctions.preprocessor import Preprocessor
from MyFunctions.searchEngine import SearchEngine, AdvancedSearchEngine
import requests
from bs4 import BeautifulSoup
import re
import os
import csv
import pandas as pd
import numpy as np
import json
from collections import defaultdict
import math
from nltk import word_tokenize
from nltk.stem import PorterStemmer


# <strong> Data collection

## <strong> 1.1 Get the list of Michelin restaraunts

Before scraping all the restaurant URLs, let's first determine the maximum page number. It's simple to find the correct CSS selector for the page list: just inspect the list of pages in your browser and identify the corresponding class or element name.

<p>
    <img title = "list of pages" src="./images/pages_number.png"/>
</p>

In [2]:
response = requests.get('https://guide.michelin.com/en/it/restaurants')
soup = BeautifulSoup(response.content, "html.parser")
page_links = soup.select('ul.pagination li a') #name of the pages list
page_numbers = [int(a.get_text()) for a in page_links if a.get_text().isdigit()]

# Get the maximum page number
total_pages = max(page_numbers) if page_numbers else 0
print(f'There are in total: {total_pages} pages')

There are in total: 100 pages


Now we can very easily get the URL of each page

In [3]:
pages = ['https://guide.michelin.com/en/it/restaurants'] #Initial page

for i in range(2, total_pages+1): #get all other pages from 2 to total_pages included
    pages.append('https://guide.michelin.com/en/it/restaurants/page/'+str(i))

Now in order to get the URLs of all the restaurants, we proceed the same by identifying the name of the corresponding class in the webpage.

<p>
<img title = "Class of a restaraunt" src="images/restaurant_link.png"/>
</p>

We can clearly see that the restaurant URLs follow a consistent pattern, which can be expressed using the regular expression:

```bash
BASE_URL/en/region/city/restaurant/name_of_restaurant
```


In [4]:
total_urls = [] #save all urls
base = 'https://guide.michelin.com' #base url to use

In [5]:
for p in pages: #loop all pages
    response = requests.get(p) #get the page
    soup = BeautifulSoup(response.content, "html.parser") # we use BeautifulSoup to get the content
    links = soup.select('a.link') #select all the class 'a link'
    pattern = re.compile(r'^/en/[^/]+/[^/]+/restaurant/[^/]+$') #pattern of restaurants
    restaurant_links = [base+link.get('href') for link in links if pattern.match(link.get('href', ''))] #get all the restaurants links
    total_urls.append(restaurant_links)

Now we save all the urls inside a txt called 'restaurant_urls.txt'

In [ ]:
with open('dataset/restaurant_urls.txt', 'w') as f: 
    page_count = 1  # Initialize the page count
    for urls in total_urls:
        f.write(f'{page_count}\n')  # Add a label for the page number
        for url in urls: # Write each URL from the current page
            f.write(f'{url}\n')  
        
        page_count += 1 # Increment the page count

In [11]:
print(sum([len(u) for u in total_urls])) # how many restaurants we got

1983


## <strong> 1.2. Crawl Michelin restaurant pages

Now we download all the HTML from the urls and save them in a folder and divide each of them in separate folder_pages

In [ ]:
crawler = Crawler()
crawler.save_all_as_html('dataset/restaurant_urls.txt') # See actual implementation inside 'crawler.py'

In [3]:
path = 'restaurants_html'
count = crawler.count_files(path)
print('file count:', count)

file count: 1983


The save_all_as_html function utilizes multi-threading to achieve optimal performance, generating approximately 20 threads concurrently. Within each loop for a page, each thread is tasked with downloading around a single URL, making it extremely efficient. Consequently, the function successfully downloaded 2,034 out of 2,037 files in under one minute.

## <strong> 1.3 Parse downloaded pages

The list of the information we desire for each restaurant and their format is as follows:

    Restaurant Name (to save as restaurantName): string;
    Address (to save as address): string;
    City (to save as city): string;
    Postal Code (to save as postalCode): string;
    Country (to save as country): string;
    Price Range (to save as priceRange): string;
    Cuisine Type (to save as cuisineType): string;
    Description (to save as description): string;
    Facilities and Services (to save as facilitiesServices): list of strings;
    Accepted Credit Cards (to save as creditCards): list of strings;
    Phone Number (to save as phoneNumber): string;
    URL to the Restaurant Page (to save as website): string.


To parse those information we can just inspect one html to see how those information are stored as we did before.<br>
Most of the information can be retrieved in the following json script at the end of each HTML file:
```js
<script type="application/ld+json">{"@context":"http://schema.org","address":{"@type":"PostalAddress","streetAddress":"Piazza Salvo d'Acquisto 16","addressLocality":"Lamezia Terme","postalCode":"88046","addressCountry":"ITA","addressRegion":"Calabria"},"name":"Abbruzzino Oltre","image":"https://axwwgrkdco.cloudimg.io/v7/__gmpics3__/f19d37d6b9da437fa06b6f9406645056.jpg?width=1000","@type":"Restaurant","review":{"@type":"Review","datePublished":"2024-09-11T07:32","name":"Abbruzzino Oltre","description":"This restaurant, the new home of young chef Luca Abbruzzino, occupies the first floor of a historic palazzo in the town centre which has recently been converted into a small hotel offering six ...","author":{"@type":"Person","name":"Michelin Inspector"}},"telephone":"+39 0968 188 8038","knowsLanguage":"en-IT","acceptsReservations":"No","servesCuisine":"Contemporary","url":"https://guide.michelin.com/en/calabria/lamezia-terme/restaurant/abbruzzino-oltre","currenciesAccepted":"EUR","paymentAccepted":"American Express credit card, Credit card / Debit card accepted, Mastercard credit card, Visa credit card","award":"Selected: Good cooking","brand":"MICHELIN Guide","hasDriveThroughService":"False","latitude":38.9770969,"longitude":16.3202202,"hasMap":"https://www.google.com/maps/search/?api=1&query=38.9770969%2C16.3202202"}</script>
```

<img src = "images/restaurant_page.png" />

Now we create a parse_restaurant function that given a html, it parses all the information we need and returns it as a dictionary, we also decided to keep region as an extra column

In [2]:
parser = Parser()
info = parser.parse_restaurant('restaurants_html/1/la-trattoria-enrico-bartolini.html') #Test
parser.show_restaurant_info(info)

restaurantName: La Trattoria Enrico Bartolini
address: Località Badiola
city: Castiglione della Pescaia
postalCode: 58043
country: ITA
region: Tuscany
priceRange: €€€€
cuisineType: Mediterranean Cuisine, Grills
description: After a majestic picture-postcard approach via a long avenue lined with cypress trees and maritime pines, passing vineyards and Maremma cattle along the way, you finally arrive at this restaurant which serves trattoria-style cuisine full of intense, familiar and reassuring flavours. The decor here is elegant with the occasional rustic touch, while the service is of the highest level yet pleasantly friendly and informal. Welcome to Bartolini’s Maremma restaurant! Here, resident chef Bruno De Moura Cossio offers a choice of dishes with one common denominator, namely charcoal grilling. All the dishes served here have been grilled in some way, so that they have a distinctive barbecued flavour. However, although the chef’s Brazilian origins are obvious in many different 

Now we can create a tsv file with all the informations of all the restaurants

In [42]:
root = 'restaurants_html'
output= 'dataset/restaurant_info.tsv'
parser.save_all_restaurant_info_to_tsv(root, output) #actual implementation in Parser class

Data saved to dataset/restaurant_info.tsv


In [43]:
df = pd.read_table('dataset/restaurant_info.tsv', index_col=0)

# <strong> Search Engine </strong>

## <strong> 2.0.0. Preprocessing the Text

Before building the search engine, we need to prepare and clean the restaurant descriptions in our dataset. To accomplish this, we created a class named Preprocessor in preprocessor.py. This class leverages the nltk library to process the text in the description column. It removes stopwords and punctuation, converts the text to lowercase, and applies stemming to reduce words to their base forms. This preprocessing step ensures that the descriptions are standardized, making them more suitable for efficient search and retrieval.

In [44]:
df = pd.read_table('dataset/restaurant_info.tsv')
preprocessor = Preprocessor()
df = preprocessor.filter(df)


[nltk_data] Downloading package stopwords to /home/pavka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/pavka/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Now let's drop the column of description as we don't need it anymore and save the result

In [ ]:
df.drop(columns='description',inplace=True)
df.to_csv('dataset/restaurant_info_filtered.tsv', sep="\t")

## <strong> 2.1. Conjunctive Query

In [6]:
df = pd.read_table("dataset/restaurant_info_filtered.tsv", index_col=0)

## <strong> 2.1.1. Create Your Index!

Let's create a vocabulary that maps each word to a unique integer (term_id) and save it in a csv file.

In [3]:
all_descriptions = df['description_filtered'].str.cat(sep=' ')
all_descriptions = list(set(all_descriptions.split(" ")))
vocabulary = {word:id for id, word in enumerate(all_descriptions)}

In [4]:
# Save this vocabulary to a file with utf-8 encoding in order to be able to handle all the characters
with open('dataset/vocabulary.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['word', 'term_id'])  # Header
    for word, id in vocabulary.items():
        writer.writerow([word, id])

Now let's create an inverted index and save it into a file.

In [66]:
vocabulary = pd.read_csv("dataset/vocabulary.csv")
df = pd.read_table("dataset/restaurant_info_filtered.tsv")
df = df[['restaurantName','description_filtered']]
df['description_filtered'] = df['description_filtered'].str.split(" ")


In [ ]:
word_to_restaurants = defaultdict(list)

# Iterate through the restaurant descriptions and update the dictionary
for i, row in df.iterrows():
    for word in row['description_filtered']:
        word_to_restaurants[word].append(row['restaurantName'])

# Add the 'restaurants_containing_word' column to the vocabulary DataFrame
vocabulary['restaurants_containing_word'] = vocabulary['word'].apply(lambda x: word_to_restaurants[x])

In [69]:
vocabulary

,word,term_id,restaurants_containing_word
0,mattiacci,0,[La Gioconda]
1,basil,1,"[Il Buco, Aria, Regallo, La Caravella dal 1959..."
2,betray,2,[Albergaccio di Castellina]
3,alway,3,"[Dal Pescatore, Osteria dell'Arco, Marcelin, A..."
4,mascia,4,[San Domenico]
...,...,...,...
7780,discreetli,7780,"[Morelli, Marelet, GioEle]"
7781,peopl,7781,"[Hosteria Giusti, Franceschetta 58, Taverna Ro..."
7782,sebada,7782,[Sa Domu Sarda]
7783,florian,7783,"[Umberto De Martino, Castel fine dining]"


Save inverted index into a file 

In [72]:
inverted_index = {term_id:rs for term_id, rs in zip(vocabulary['term_id'], vocabulary['restaurants_containing_word'])}
with open('dataset/inverted_index.json', 'w') as jsonfile:
    json.dump(inverted_index, jsonfile)

## <strong> 2.1.2. Execute the Query

In [3]:
originale = "dataset/restaurant_info.tsv"
df = "dataset/restaurant_info_filtered.tsv"
vocabulary = "dataset/vocabulary.csv"
inverted_index = "dataset/inverted_index.json"

searcher = SearchEngine(originale, df,vocabulary,inverted_index)

In [4]:
ideal_restaurants = searcher.search("greek")
ideal_restaurants

,restaurantName,address,description,website
193,Hyle,contrada Torre Garga SS 107,Located on the wooded Sila plateau famous sinc...,https://www.hyleristorante.it/
269,Eea,via Umberto I,"Named after the Ancient Greek name for Ponza, ...",https://www.mondoeea.it
985,Taverna Kerkira,corso Vittorio Emanuele 217,Kerkira (Corfu in Greek) owes its existence to...,NaN
1223,50 Kalò,piazza Sannazzaro 201/b,In the Neapolitan tradition of interpreting dr...,https://www.50kalo.it
1484,Il Pievano,Località Spaltenna 13,"Occupying the charming, romantic setting of a ...",https://www.spaltenna.it/it/ristoranti/il-pievano
1814,Palais Royal Restaurant,Calle Larga 22 Marzo 2032,This restaurant is an Italian offshoot of the ...,https://nolinskivenezia.com/it/palais-royal-re...


# <strong> 2.2. Ranked Search Engine with TF-IDF and Cosine Similarity

## <strong> 2.2.1 Inverted Index with TF-IDF Scores

Let's combine all the functions above in order to calculate the TF, IDF, and TF-IDF scores, and store the updated inverted index.

In [5]:
query_scores = searcher.get_restaurant_scores("japanese")

In [8]:
query_scores 

[('Seta Sushi Restaurant', np.float64(0.47957404597332215)),
 ('Saffron', np.float64(0.42802817441699037)),
 ('Ichikawa', np.float64(0.4163068586715535)),
 ('Zero Milano', np.float64(0.41046809035207754)),
 ('Hazama', np.float64(0.40495991472950993)),
 ('Sushisen', np.float64(0.38968773824132646)),
 ("Wicky's Innovative Japanese Cuisine", np.float64(0.378731173891894)),
 ('J Contemporary Japanese Restaurant', np.float64(0.2790185249476035)),
 ('Nobuya', np.float64(0.2747172620764188)),
 ('Nida', np.float64(0.2592283288861831)),
 ('Serendepico', np.float64(0.24151074310401682)),
 ('Kandoo Nippon Restaurant', np.float64(0.23232346564116108)),
 ('Chat Qui Rit', np.float64(0.22223694544328512)),
 ('Iyo', np.float64(0.21433365571769658)),
 ("Enoteca L'Armadillo", np.float64(0.17150657300297373)),
 ('Hidalgo', np.float64(0.1708622256182522)),
 ('Shiroya', np.float64(0.16562480155330214)),
 ('Urubamba', np.float64(0.1628117816468493)),
 ('Kohaku', np.float64(0.1513712806970587)),
 ('Boivin', 

## <strong> 2.2.2. Execute the Ranked Query

# <strong> 3. Define a New Score!

 <strong> Steps:<br>
    - User Query: The user provides a text query. We’ll retrieve relevant documents using the search engine built in Step 2.1.<br>
    - New Ranking Metric: After retrieving relevant documents, we’ll rank them using a new custom score. Instead of limiting the scoring to only the description field, we can include other attributes like priceRange, facilitiesServices, and cuisineType.<br>
    - You will use a heap data structure (e.g., Python’s heapq library) to maintain the top-k restaurants.


In [3]:
df = pd.read_table("dataset/restaurant_info.tsv")

# <strong> 5. BONUS: Advanced Search Engine

## <strong> 5.1 - Specify Search Criteria: Users can specify search terms for the following features (any or all of them):
    - restaurantName
    - city
    - cuisineType


Let's first create the inverted index for each of the features, we can do it very easily by creating a function and repeat the function call for each column we want.


In [4]:
df = pd.read_table("dataset/restaurant_info.tsv")

In [40]:
def createIndex(df, column):
    # Convert all unique words to lowercase and apply stemming
    stemmer = PorterStemmer()
    df[column] = df[column].fillna('na')
    all_words = pd.Series(" ".join(df[column].str.lower()).split()).unique()
    all_words_stemmed = [stemmer.stem(word) for word in all_words]
    
    # Initialize the dictionary for stemmed words
    word_to_restaurants = {word: [] for word in all_words_stemmed}
    
    # Split each row's text, stem each word, and update the index
    for i, row in df.iterrows():
        words = row[column].lower().split()
        for word in words:
            stemmed_word = stemmer.stem(word)
            word_to_restaurants[stemmed_word].append(row['restaurantName'])
    
    # Save to JSON
    path = f'dataset/{column}_index.json'
    with open(path, 'w') as jsonfile:
        json.dump(word_to_restaurants, jsonfile)

In [ ]:
createIndex(df, 'restaurantName')
createIndex(df, 'city')
createIndex(df, 'cuisineType')

### <strong> 5.2 - Price Range Filter: Allow users to set a price range (e.g., between € and €€€) to filter the results by affordability.

In [29]:
createIndex(df, 'priceRange')

### <strong> 5.3 - Region Filter: Enable users to specify a list of Italian regions to limit the search to restaurants within those regions.

In [30]:
createIndex(df, 'region')

### <strong> 5.4 - Accepted Credit Cards: Provide an option to filter by accepted credit card types. Users can specify one or more preferred card types (e.g., Visa, MasterCard, Amex).

In [41]:
createIndex(df, 'creditCards')

### <strong> 5.5 - Services and Facilities: Allow users to filter based on specific services and facilities provided by the restaurant. For example, users may look for amenities like Wi-Fi, Terrace, Air Conditioning, or Parking.

In [42]:
createIndex(df, 'facilitiesServices')

## <strong> Now we can just implement use the AdvancedSearchEngine

In [9]:
advanced = AdvancedSearchEngine()
query = dict(
    city = 'catania',
    region = 'sicily'
)
advanced.search(**query)

,restaurantName,address,cuisineType,priceRange,website
12,Angiò-Macelleria di Mare,viale Africa 28/h,Seafood,€€,https://albertoangiolucci.it/
563,Sapio,piazza Antonino Gandolfo 11,"Sicilian, Modern Cuisine",€€€€,https://www.sapiorestaurant.it/
996,Me Cumpari Turiddu,piazza Turi Ferro 36,Sicilian,€,https://www.mecumparituriddu.it/
1054,Ménage,Via Euplio Reina 13,"Sicilian, Contemporary",€€,https://www.menagelounge.it/
1550,Concezione Restaurant,via Giuseppe Verdi 143,"Creative, Sicilian",€€€,https://concezionerestaurant.com/
1779,Materia | Spazio Cucina,via Teatro Massimo 29,"Sicilian, Modern Cuisine",€€,https://www.materiaspaziocucina.it/
1934,Coria,Via Prefettura 21,"Italian Contemporary, Sicilian",€€€,https://www.ristorantecoria.it
